# Biostat 257 Homework 4

**Due May 20 @ 11:59PM**

In [1]:
versioninfo()

Julia Version 1.7.2
Commit bf53498635 (2022-02-06 15:21 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin19.5.0)
  CPU: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, skylake)


We are going to try different numerical methods learnt in class on the [Google PageRank problem](https://en.wikipedia.org/wiki/PageRank).

## Q1 (5 pts) Recognize structure

Let $\mathbf{A} \in \{0,1\}^{n \times n}$ be the connectivity matrix of $n$ web pages with entries
$$
\begin{eqnarray*}
	a_{ij}= \begin{cases}
	1 & \text{if page $i$ links to page $j$} \\
	0 & \text{otherwise}
	\end{cases}.
\end{eqnarray*}
$$
$r_i = \sum_j a_{ij}$ is the out-degree of page $i$. That is $r_i$ is the number of links on page $i$. Imagine a random surfer exploring the space of $n$ pages according to the following rules.  

- From a page $i$ with $r_i>0$
    * with probability $p$, (s)he randomly chooses a link on page $i$ (uniformly) and follows that link to the next page  
    * with probability $1-p$, (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
- From a page $i$ with $r_i=0$ (a dangling page), (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
    
The process defines a Markov chain on the space of $n$ pages. Write the transition matrix $\mathbf{P}$ of the Markov chain as a sparse matrix plus rank 1 matrix.

**Solution**

$$
P = \text{diag}(d)A + a1'
$$

where $d$ is a $n \times 1$ vector, with 
$$
d_i = \begin{cases}
\frac{p}{r_i}, & r_i > 0, \\
0, & r_i = 0 
\end{cases}
$$
and $a$ is a $n \times 1$ vector, with 
$$
a_i = \begin{cases}
\frac{1-p}{n}, & r_i > 0, \\
\frac{1}{n}, & r_i = 0 
\end{cases}
$$

A is a sparse matrix, and $a1'$ is a rank 1 matrix. 

## Q2 Relate to numerical linear algebra

According to standard Markov chain theory, the (random) position of the surfer converges to the stationary distribution $\mathbf{x} = (x_1,\ldots,x_n)^T$ of the Markov chain. $x_i$ has the natural interpretation of the proportion of times the surfer visits page $i$ in the long run. Therefore $\mathbf{x}$ serves as page ranks: a higher $x_i$ means page $i$ is more visited. It is well-known that $\mathbf{x}$ is the left eigenvector corresponding to the top eigenvalue 1 of the transition matrix $\mathbf{P}$. That is $\mathbf{P}^T \mathbf{x} = \mathbf{x}$. Therefore $\mathbf{x}$ can be solved as an **eigen-problem**. It can also be cast as **solving a linear system**. Since the row sums of $\mathbf{P}$ are 1, $\mathbf{P}$ is rank deficient. We can replace the first equation by the $\sum_{i=1}^n x_i = 1$.

Hint: For iterative solvers, we don't need to replace the 1st equation. We can use the matrix $\mathbf{I} - \mathbf{P}^T$ directly if we start with a vector with all positive entries.

## Q3 (10 pts) Explore data

Obtain the connectivity matrix `A` from the `SNAP/web-Google` data in the MatrixDepot package. 

In [2]:
using MatrixDepot

md = mdopen("SNAP/web-Google")
# display documentation for the SNAP/web-Google data
mdinfo(md)

┌ Info: verify download of index files...
└ @ MatrixDepot /Users/julietzhou/.julia/packages/MatrixDepot/GEDc3/src/MatrixDepot.jl:139
┌ Info: reading database
└ @ MatrixDepot /Users/julietzhou/.julia/packages/MatrixDepot/GEDc3/src/download.jl:23
┌ Info: adding metadata...
└ @ MatrixDepot /Users/julietzhou/.julia/packages/MatrixDepot/GEDc3/src/download.jl:67
┌ Info: adding svd data...
└ @ MatrixDepot /Users/julietzhou/.julia/packages/MatrixDepot/GEDc3/src/download.jl:69
┌ Info: writing database
└ @ MatrixDepot /Users/julietzhou/.julia/packages/MatrixDepot/GEDc3/src/download.jl:74
┌ Info: used remote sites are sparse.tamu.edu with MAT index and math.nist.gov with HTML index
└ @ MatrixDepot /Users/julietzhou/.julia/packages/MatrixDepot/GEDc3/src/MatrixDepot.jl:141


# SNAP/web-Google

###### MatrixMarket matrix coordinate pattern general

---

  * UF Sparse Matrix Collection, Tim Davis
  * http://www.cise.ufl.edu/research/sparse/matrices/SNAP/web-Google
  * name: SNAP/web-Google
  * [Web graph from Google]
  * id: 2301
  * date: 2002
  * author: Google
  * ed: J. Leskovec
  * fields: name title A id date author ed kind notes
  * kind: directed graph

---

  * notes:
  * Networks from SNAP (Stanford Network Analysis Platform) Network Data Sets,
  * Jure Leskovec http://snap.stanford.edu/data/index.html
  * email jure at cs.stanford.edu
  * 
  * Google web graph
  * 
  * Dataset information
  * 
  * Nodes represent web pages and directed edges represent hyperlinks between them.
  * The data was released in 2002 by Google as a part of Google Programming
  * Contest.
  * 
  * Dataset statistics
  * Nodes   875713
  * Edges   5105039
  * Nodes in largest WCC    855802 (0.977)
  * Edges in largest WCC    5066842 (0.993)
  * Nodes in largest SCC    434818 (0.497)
  * Edges in largest SCC    3419124 (0.670)
  * Average clustering coefficient  0.6047
  * Number of triangles     13391903
  * Fraction of closed triangles    0.05523
  * Diameter (longest shortest path)    22
  * 90-percentile effective diameter    8.1
  * 
  * Source (citation)
  * 
  * J. Leskovec, K. Lang, A. Dasgupta, M. Mahoney. Community Structure in Large
  * Networks: Natural Cluster Sizes and the Absence of Large Well-Defined Clusters.
  * arXiv.org:0810.1355, 2008.
  * 
  * Google programming contest, 2002
  * http://www.google.com/programming-contest/
  * 
  * Files
  * File    Description
  * web-Google.txt.gz   Webgraph from the Google programming contest, 2002

---

916428 916428 5105039


In [3]:
# connectivity matrix
A = md.A

916428×916428 SparseArrays.SparseMatrixCSC{Bool, Int64} with 5105039 stored entries:
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿

Compute summary statistics:
* How much memory does `A` take? If converted to a `Matrix{Float64}` (don't do it!), how much memory will it take?  
* number of web pages
* number of edges (web links). 
* number of dangling nodes (pages with no out links)
* histogram of in-degrees  
* list the top 20 pages with the largest in-degrees?  
* histogram of out-degrees
* which the top 20 pages with the largest out-degrees?
* visualize the sparsity pattern of $\mathbf{A}$ or a submatrix of $\mathbf{A}$ say `A[1:10000, 1:10000]`. 

**Hint**: For plots, you can use the [UnicodePlots.jl](https://github.com/Evizero/UnicodePlots.jl) package (`spy`, `histogram`, etc), which is fast for large data. 

**Solution**

In [4]:
# memory in bytes 
Base.summarysize(A)

53276943

In [5]:
# approximate size of dense A in bytes
size(A)[1]^2 * 8

6718722233472

In [6]:
# number of web pages = 916428
n = size(A, 1)

916428

In [7]:
# number of edges
outdeg = sum(A, dims = 2)
sum(outdeg)

5105039

In [8]:
# number of dangling nodes
sum(iszero, outdeg)

176974

In [9]:
using UnicodePlots
# histogram of in-degrees
indeg = sum(A, dims = 1)
# UnicodePlots.histogram(indeg, closed = :right, nbins = 20) # very right skewed
UnicodePlots.histogram(indeg, xscale = log10, closed = :left, nbins = 30)

                    ┌                                        ┐ 
   [   0.0,  200.0) ┤████████████████████████████████  915169  
   [ 200.0,  400.0) ┤███████████████▋ 812                      
   [ 400.0,  600.0) ┤████████████▍ 203                         
   [ 600.0,  800.0) ┤██████████▎ 80                            
   [ 800.0, 1000.0) ┤███████▉ 30                               
   [1000.0, 1200.0) ┤██████▊ 19                                
   [1200.0, 1400.0) ┤█████▊ 12                                 
   [1400.0, 1600.0) ┤████▎ 6                                   
   [1600.0, 1800.0) ┤████▎ 6                                   
   [1800.0, 2000.0) ┤█████▎ 9                                  
   [2000.0, 2200.0) ┤████▊ 8                                   
   [2200.0, 2400.0) ┤███▎ 4                                    
   [2400.0, 2600.0) ┤████▌ 7                                   
   [2600.0, 2800.0) ┤█████▋ 11                                 
   [2800.0, 3000.0) ┤████▎ 6            

In [10]:
sortperm(vec(indeg), rev = true)[1:20]

20-element Vector{Int64}:
 537040
 597622
 504141
 751385
  32164
 885606
 163076
 819224
 605857
 828964
 551830
  41910
 558792
 459075
 407611
 213433
 765335
 384667
 173977
 687326

In [11]:
# histogram of out-degrees
UnicodePlots.histogram(outdeg, xscale = log10, closed = :left, nbins = 30)

                  ┌                                        ┐ 
   [  0.0,  20.0) ┤████████████████████████████████  891798  
   [ 20.0,  40.0) ┤███████████████████████▌ 22628            
   [ 40.0,  60.0) ┤████████████████▊ 1329                    
   [ 60.0,  80.0) ┤█████████████▊ 371                        
   [ 80.0, 100.0) ┤███████████▍ 124                          
   [100.0, 120.0) ┤██████████▍ 86                            
   [120.0, 140.0) ┤███████▊ 29                               
   [140.0, 160.0) ┤██████▍ 15                                
   [160.0, 180.0) ┤██████▍ 15                                
   [180.0, 200.0) ┤████▊ 8                                   
   [200.0, 220.0) ┤████▌ 7                                   
   [220.0, 240.0) ┤██▌ 3                                     
   [240.0, 260.0) ┤████▊ 8                                   
   [260.0, 280.0) ┤██▌ 3                                     
   [280.0, 300.0) ┤  0                                       
   [300.

In [12]:
sortperm(vec(outdeg), rev = true)[1:20]

20-element Vector{Int64}:
 506743
 203749
 305230
 768092
 808644
 412411
 600480
 376429
 156951
 885729
 667585
 685696
 282141
 598189
 579315
 411594
 321092
 838279
 302734
 915274

In [13]:
using UnicodePlots
spy(A[1:10000, 1:10000])

         ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀Sparsity Pattern⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀    
         ┌──────────────────────────────────────────┐    
       1 │⠀⠀⠀⠀⠀⠀⠀⠂⠀⠀⠀⠀⢀⢂⡂⠐⠀⠀⠀⠠⠀⠀⠀⠔⠄⠀⠈⢀⠀⠀⠉⠀⠀⠁⠀⠀⠐⠀⠄⠀⠐⠀│ > 0
         │⠀⠀⠀⠀⠀⠂⡂⠀⠄⠄⠈⠈⡔⠀⠀⠁⠂⠀⠀⠀⠀⠠⠁⠒⠁⠀⠄⠀⠐⠀⠈⠀⠀⡐⠀⠀⠠⠀⡀⠀⠉⠀│ < 0
         │⠀⠐⠠⠀⠀⠁⢐⠄⠀⠢⠠⠀⠀⡀⠈⠀⠋⠀⠀⠀⠀⠀⠂⡀⠀⠈⠄⠀⠄⠁⠈⠀⠠⠀⠰⠀⠀⠀⠀⢀⠆⠀│    
         │⠀⠈⠠⠂⠀⠈⠀⡠⠐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠈⠄⠁⢀⡀⠐⠄⠄⠀⠀⢀⠀⠈⠂⢠⠀⠢⠐⠀⠀⠀⡀│    
         │⠀⠀⠐⡁⠀⡀⠀⠀⠠⠀⠁⠀⠀⠀⡄⠀⠈⠁⠀⠀⠀⠀⠠⠂⠀⠀⠀⠀⠀⠂⠪⠀⠀⠂⠀⠀⠀⠄⢀⠀⠁⠀│    
         │⠂⢀⠀⠁⠀⠀⡀⡉⠀⠀⠠⠂⠐⢀⠄⠁⠐⠀⠈⠀⠀⠀⠀⠀⡀⠐⠁⡠⠀⠂⠚⠀⠁⠀⠐⢀⠆⠀⠈⠄⢀⠂│    
         │⠀⠀⠀⠁⠀⡄⠀⠀⠀⠀⠒⠀⢄⠄⢂⠀⠂⠀⠀⠄⠀⢀⠀⠣⠀⢀⠀⠀⠀⠀⠠⡀⠀⠀⢠⠄⠀⡀⠀⠂⠂⠀│    
         │⢀⠈⠀⠀⠂⠀⠀⠁⢠⠈⠄⠁⠀⠀⡈⠈⠠⠀⢀⡦⠀⠀⠅⠐⠀⠨⠁⠨⠀⠄⠠⢀⠀⠐⢐⠀⠀⠉⠀⠀⠠⠉│    
         │⠀⠀⠨⠀⠋⠀⠄⠀⠂⠀⠈⠀⠀⠀⠈⠀⠀⠀⠀⡑⠀⢀⠈⠈⠀⠀⠀⠈⠀⠂⠐⠼⠀⠠⠀⠀⡈⠁⠀⠀⠡⠄│    
         │⠠⡁⠀⠀⠀⠀⠀⠀⠀⠀⠂⢀⠀⠀⠀⡄⠀⠀⠀⠤⠀⠀⠁⠀⡂⠀⠀⠀⠄⠀⢈⠀⢔⡊⣨⠈⠂⠀⠄⢀⠀⠀│    
         │⠀⠂⠀⠈⠀⠀⠀⠀⠀⠀⠈⠀⡀⠀⠂⠠⠐⠀⠄⠀⠀⠀⠀⠀⠀⠤⠀⠀⠀⠠⠘⠐⠀⠄⢘⠀⠀⢐⠄⠀⠁⠀│    
         │⢀⠄⢠⠄⡊⠂⠢⢀⠀⠀⠀⠀⠄⡀⡑⠀⡐⠀⠁⡂⠀⠀⠀⠰⠂⢠⠄⠈⠀⠈⠐⠀⠀⠀⠉⠀⠀⡀⠀⠀⠃⠀│    
         │⠀⠀⠅⠀⡁⠀⠀⠀⠀⠀⢁⢀⠀⡀⡀⡐⠀⠀⠈⠈⠔⠄⠀⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠄⢉⠠⠁⠀⠁⠀⠀⠀│    
         │⠊⢈⠀⠁⠀⠀⠀⠅⠀⠂⠀⡀⠀⠀⠀⠀⡀⠀⠀⠐⠄⠁⠠⠀⠐⠈⠀⠀⠀⡀⢠⠀⠀⠀⢰⠀⠀⠀⢀⠀⠠⠂│    
         │⠈⠀⠀⠈⠄⠀⠀⠄⠐⠀⠠⠁⠁⠀⡂⠀⠠⠄⠌⠅⣀⠀⠀⢀⠀⠠⠀⠠⠠⠀⠲⢂⠀⠰⡄⠀⢀⠬⠈⠀⠀⠀│    
         │⠈⠀⠀⠄

## Q4 (5 pts) Dense linear algebra? 

Consider the following methods to obtain the page ranks of the `SNAP/web-Google` data. 

1. A dense linear system solver such as LU decomposition.  
2. A dense eigen-solver for asymmetric matrix.  

For the LU approach, estimate (1) the memory usage and (2) how long it will take assuming that the LAPACK functions can achieve the theoretical throughput of your computer. 

**Solution**

In [14]:
# # of flops if solving with LU decomposition 
flops = 2 / 3 * n^3 + 2 * n^2

5.131037779285815e17

In [15]:
# time (in days) assuming my computer can do 10^11 flops per second
flops / 10^11 / 60 / 60 / 24

59.38701133432656

## Q5 (75 pts) Iterative solvers

Set the _teleportation_ parameter at $p = 0.85$. Consider the following methods for solving the PageRank problem. 

1. An iterative linear system solver such as GMRES. 
2. An iterative eigen-solver such as Arnoldi method.

For iterative methods, we have many choices in Julia. See a list of existing Julia packages for linear solvers at this [page](https://jutho.github.io/KrylovKit.jl/stable/#Package-features-and-alternatives-1). The start-up code below uses the [KrylovKit.jl](https://github.com/Jutho/KrylovKit.jl) package. You can use other packages if you prefer. Make sure to utilize the special structure of $\mathbf{P}$ (sparse + rank 1) to speed up the matrix-vector multiplication. 

### Step 1 (15 pts)

Let's implement a type `PageRankImPt` that mimics the matrix $\mathbf{M} = \mathbf{I} - \mathbf{P}^T$. For iterative methods, all we need to provide are methods for evaluating $\mathbf{M} \mathbf{v}$ and $\mathbf{M}^T \mathbf{v}$ for arbitrary vector $\mathbf{v}$.

In [16]:
using BenchmarkTools, LinearAlgebra, SparseArrays, Revise

# a type for the matrix M = I - P^T in PageRank problem
struct PageRankImPt{TA <: Number, IA <: Integer, T <: AbstractFloat} <: AbstractMatrix{T}
    A         :: SparseMatrixCSC{TA, IA} # adjacency matrix
    telep     :: T
    # working arrays
    # TODO: whatever intermediate arrays you may want to pre-allocate
    n         :: IA
    r         :: Matrix{IA}
    d         :: Vector{T}
    a         :: Vector{T}
    storage_n :: Vector{T}
end

# constructor
function PageRankImPt(A::SparseMatrixCSC, telep::T) where T <: AbstractFloat
    n = size(A, 1)
    # TODO: initialize and pre-allocate arrays
    r = sum(A, dims = 2)
    d = Vector{T}(undef, n)
    a = Vector{T}(undef, n)
    for i in 1:n
        if r[i, 1] > 0 
            d[i] = telep / r[i, 1]
            a[i] = (1 - telep) / n
        else 
            d[i] = 0
            a[i] = 1 / n
        end
    end
    
    storage_n = similar(d)
         
    PageRankImPt(A, telep, n, r, d, a, storage_n)
end

LinearAlgebra.issymmetric(::PageRankImPt) = false
Base.size(M::PageRankImPt) = size(M.A)
# TODO: implement this function for evaluating M[i, j]
Base.getindex(M::PageRankImPt, i, j) = 1 - M.d[j] * M.A[j, i] - M.a[j]

To simplify the two multiplications, we observe that
$$
(I - P') v = v - A' \text{diag}(d) v - 1a'v
$$
and 
$$
(I - P) v = v - \text{diag}(d) A v - a1'v
$$

In [17]:
# overwrite `out` by `(I - Pt) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        M   :: PageRankImPt{<:Number, <:Integer, T}, 
        v   :: Vector{T}
        ) where T <: AbstractFloat
    # TODO: implement mul!(out, M, v)
    
    # M.storage_n = diag(d) * v
    M.storage_n .= M.d .* v
    # out = A' * M.storage_n
    mul!(out, transpose(M.A), M.storage_n)

    out .= v .- out .- dot(v, M.a)
    
    return out
end

# overwrite `out` by `(I - P) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        Mt  :: Transpose{T, PageRankImPt{TA, IA, T}}, 
        v   :: Vector{T}
        ) where {TA<:Number, IA<:Integer, T <: AbstractFloat}
    M = Mt.parent
    # TODO: implement mul!(out, transpose(M), v)
    
    # out = A * v
    mul!(out, M.A, v)
    # out = diag(d) * out
    out .= M.d .* out
    
    out .= v .- out .- M.a .* sum(v)
    
    return out
end

To check correctness. Note 
$$
\mathbf{M}^T \mathbf{1} = \mathbf{0}
$$
and
$$
\mathbf{M} \mathbf{x} = \mathbf{0}
$$
for stationary distribution $\mathbf{x}$.

Download the solution file `pgrksol.csv.gz`. **Do not put this file in your Git**. You will lose points if you do. You can add a line `pgrksol.csv.gz` to your `.gitignore` file.

In [18]:
using CodecZlib, DelimitedFiles

isfile("pgrksol.csv.gz") || download("https://raw.githubusercontent.com/ucla-biostat-257/2022spring/master/hw/hw4/pgrksol.csv.gz")
xsol = open("pgrksol.csv.gz", "r") do io
    vec(readdlm(GzipDecompressorStream(io)))
end

916428-element Vector{Float64}:
 3.3783428216975054e-5
 2.0710155392568165e-6
 3.663065984832893e-6
 7.527510785028837e-7
 8.63328599674051e-7
 1.769418252415541e-6
 2.431230382883396e-7
 6.368417180141445e-7
 4.744973703681939e-7
 2.6895486110647536e-7
 3.18574314847409e-6
 7.375106374416742e-7
 2.431230382883396e-7
 ⋮
 1.1305006040148547e-6
 4.874825281822915e-6
 3.167946973112519e-6
 9.72688040308568e-7
 6.588614479285245e-7
 7.737011774300648e-7
 2.431230382883396e-7
 1.6219204214797293e-6
 3.912130060551738e-7
 2.431230382883396e-7
 7.296033831163157e-6
 6.330939996912478e-7

**You will lose all 35 points (Steps 1 and 2)** if the following statements throw AssertError.

In [19]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)

@assert norm(transpose(M) * ones(n)) < 1e-12

In [20]:
@assert norm(M * xsol) < 1e-12

### Step 2 (20 pts)

We want to benchmark the hot functions `mul!` to make sure they are efficient and allocate little memory.

In [21]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)
v, out = ones(n), zeros(n)
bm_mv = @benchmark mul!($out, $M, $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 212 samples with 1 evaluation.
 Range (min … max):  22.052 ms … 39.987 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     22.487 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   22.965 ms ±  1.549 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▂██▄                                                        
  ▇█████▇█▆▅▄▄▄▄▃▄▂▃▂▃▁▂▂▂▁▃▁▃▃▂▁▁▂▁▂▂▃▂▁▁▂▁▁▂▁▁▂▁▁▁▁▁▂▁▃▁▁▁▂ ▃
  22.1 ms         Histogram: frequency by time        27.2 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [22]:
bm_mtv = @benchmark mul!($out, $(transpose(M)), $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 165 samples with 1 evaluation.
 Range (min … max):  28.278 ms … 33.657 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     29.205 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   29.608 ms ±  1.157 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▃▄▃▂▄▅▇█▄▂   ▅         ▂                                   
  ▆▇███████████▇▅█▆▅▅▃▇▇▁▅▆█▆▃▃▅▃▃▃▁▃▁▁▃▃▃▅▁▃▆▃▅▁▁▃▁▁▁▁▃▃▁▃▃▃ ▃
  28.3 ms         Histogram: frequency by time          33 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [23]:
clamp(10 - median(bm_mv).memory / 100, 0, 10) + 
clamp(10 - median(bm_mtv).memory / 100, 0, 10)

20.0

**Hint**: My median run times are 30-40 ms and memory allocations are 0 bytes.

### Step 3 (20 pts)

Let's first try to solve the PageRank problem by the GMRES method for solving linear equations. 

In [24]:
using KrylovKit

# normalize in-degrees to be the start point
x0   = vec(sum(A, dims = 1)) .+ 1.0
x0 ./= sum(x0)

# right hand side
b = zeros(n)

# warm up (compilation)
linsolve(M, b, x0, issymmetric = false, isposdef = false, maxiter = 1) 
# output is complex eigenvalue/eigenvector
(x_gmres, info), time_gmres, = @timed linsolve(M, b, x0, issymmetric = false, isposdef = false)

(value = ([3.3783428221888584e-5, 2.0710155392484385e-6, 3.663065985237124e-6, 7.527510785602639e-7, 8.633285997161323e-7, 1.7694182527377933e-6, 2.431230382912157e-7, 6.368417180747229e-7, 4.744973703749994e-7, 2.689548611096568e-7  …  3.1679469739776893e-6, 9.7268804102312e-7, 6.588614478542467e-7, 7.737011774714668e-7, 2.431230382912157e-7, 1.621920421424626e-6, 3.91213006063147e-7, 2.431230382912157e-7, 7.296033831324964e-6, 6.330939996678448e-7], ConvergenceInfo: one converged value after 3 iterations and 72 applications of the linear map;
norms of residuals are given by (7.821167231992103e-13,).
), time = 3.809713475, bytes = 1009034460, gctime = 0.29973655, gcstats = Base.GC_Diff(1009034460, 137, 0, 75141, 5, 0, 299736550, 4, 2))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [25]:
@assert norm(x_gmres - xsol) < 1e-8

GMRES should be reasonably fast. The points you'll get is

In [26]:
clamp(20 / time_gmres * 20, 0, 20)

20.0

**Hint**: My runtime is about 7-8 seconds.

### Step 4 (20 pts)

Let's first try to solve the PageRank problem by the Arnoldi method for solving eigen problems. 

In [27]:
# warm up (compilation)
eigsolve(M, x0, 1, :SR, issymmetric = false, maxiter = 1)
# output is complex eigenvalue/eigenvector
(vals, vecs, info), time_arnoldi, = @timed eigsolve(M, x0, 1, :SR, issymmetric = false)

(value = (ComplexF64[-8.937262196793364e-15 + 0.0im], Vector{ComplexF64}[[0.005635826953807289 + 0.0im, 0.0003454914380782886 + 0.0im, 0.0006110808494125137 + 0.0im, 0.00012557561626035183 + 0.0im, 0.0001440224053280327 + 0.0im, 0.0002951783050398468 + 0.0im, 4.055832828677895e-5 + 0.0im, 0.0001062393578485207 + 0.0im, 7.915671116195337e-5 + 0.0im, 4.48676506672492e-5 + 0.0im  …  0.0005284839899796993 + 0.0im, 0.00016226599147988675 + 0.0im, 0.00010991273837609693 + 0.0im, 0.00012907055855751663 + 0.0im, 4.055832828677895e-5 + 0.0im, 0.00027057238743203986 + 0.0im, 6.526302748333374e-5 + 0.0im, 4.055832828677895e-5 + 0.0im, 0.0012171406601328128 + 0.0im, 0.00010561415510699163 + 0.0im]], ConvergenceInfo: one converged value after 7 iterations and 99 applications of the linear map;
norms of residuals are given by (9.576431215525902e-14,).
), time = 7.668278204, bytes = 1581384437, gctime = 0.403376625, gcstats = Base.GC_Diff(1581384437, 213, 0, 77836, 20, 0, 403376625, 5, 2))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [28]:
@assert abs(Real(vals[1])) < 1e-8

In [29]:
x_arnoldi   = abs.(Real.(vecs[1]))
x_arnoldi ./= sum(x_arnoldi)
@assert norm(x_arnoldi - xsol) < 1e-8

Arnoldi should be reasonably fast. The points you'll get is

In [30]:
clamp(20 / time_arnoldi * 20, 0, 20)

20.0

**Hint**: My runtime is about 11-12 seconds.

## Q6 (5 pts) Results

List the top 20 pages you found and their corresponding PageRank score. Do they match the top 20 pages ranked according to in-degrees? 

**Solution**

The top 20 pages by PageRank score and by in-degrees are shown below. We see that the order does not match exactly, but very important pages show up in both lists. 

In [31]:
# gmres results
score_gmre = sort(x_gmres, rev = true)[1:20]
page_gmre = sortperm(x_gmres, rev = true)[1:20]
# in-degree results
indegree = sort(vec(indeg), rev = true)[1:20]
page_indeg = sortperm(vec(indeg), rev = true)[1:20]
[page_gmre score_gmre page_indeg indegree]

20×4 Matrix{Float64}:
 597622.0  0.000914581  537040.0  6326.0
  41910.0  0.000912013  597622.0  5354.0
 163076.0  0.000895056  504141.0  5271.0
 537040.0  0.000889934  751385.0  5182.0
 384667.0  0.000779103   32164.0  5097.0
 504141.0  0.000757542  885606.0  4847.0
 486981.0  0.000717764  163076.0  4731.0
 605857.0  0.000710848  819224.0  4620.0
  32164.0  0.000705518  605857.0  4550.0
 558792.0  0.000702166  828964.0  4484.0
 551830.0  0.000695075  551830.0  4220.0
 765335.0  0.000676228   41910.0  4219.0
 751385.0  0.000654656  558792.0  4206.0
 425771.0  0.000616848  459075.0  4187.0
 908352.0  0.000614622  407611.0  4180.0
 173977.0  0.000603115  213433.0  4084.0
   7315.0  0.000592664  765335.0  4015.0
 213433.0  0.000589447  384667.0  4010.0
 885606.0  0.000581266  173977.0  3988.0
 819224.0  0.000576519  687326.0  3956.0

## Q7 Be proud of yourself

Go to your resume/cv and claim you have experience performing analysis on a network of one million nodes.